# Importing Libraries

In [2]:
#importing libraries
import random
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Simulating Data

## Creating distributions

In [50]:

# creating the sample size
n = 10000
np.random.seed(33) 
#random normal variables with mean of 0 and SD of 1 (default)
x1 = np.random.normal(0, 1, n)
x2 = np.random.normal(0, 1, n)
#random uniform variables with min of 0 and max of 1 (default)
x3 = np.random.uniform(0, 1, n)
x4 = np.random.uniform(0, 1, n)

## Adding Coefficients
The coefficients were based on this study: https://digitalcommons.coastal.edu/cgi/viewcontent.cgi?article=1220&context=etd!

In [51]:
# based on this study: 
b0 = 3.87
b1 = 0.81
b2 = -0.02
b3 = 1.06
b4 = 1.65

#adding error
error = np.random.normal(0, 1, n)   

#generating the equation for y
y = (b0 + b1*x1 + b2*x2 + b3*x3 + b4*x4 + b2*(x2**2) +
     error)
#creating the data_frame
data = pd.DataFrame({'x1': x1,'x2': x2,'x3': x3,'x4': x4, 'x2_sq': x2**2, 'y': y})
print(data.head())

         x1        x2        x3        x4     x2_sq         y
0 -0.318854 -1.097509  0.441344  0.615104  1.204527  3.175933
1 -1.602981  0.704783  0.162942  0.566509  0.496719  2.547988
2 -1.535218 -0.591034  0.864158  0.650995  0.349322  5.725682
3 -0.570401  0.500857  0.806055  0.364113  0.250857  5.365241
4 -0.216728  1.280189  0.063567  0.223869  1.638883  4.450283


# Validation with 30% test and 70% training

In [69]:
#creating the training/test data 
train, test = train_test_split(data, 
                                test_size = .30,
                                random_state = 33)

In [70]:
X_train = train[['x1','x2','x3','x4','x2_sq']]
y_train = train['y']
X_test = test[['x1','x2','x3','x4','x2_sq']]
y_test = test['y']

X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

In [71]:
model = sm.OLS(y_train, X_train_sm)
results = model.fit()

In [72]:
yhat_test = results.predict(X_test_sm)
yhat_train = results.predict(X_train_sm)
MSE_test = np.mean((y_test - yhat_test)**2)
MSE_train = np.mean((y_train - yhat_train)**2)

In [77]:
print(results.summary())
print("Training MSE:", MSE_train)
print("Test MSE:", MSE_test)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.501
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     1405.
Date:                Mon, 15 Sep 2025   Prob (F-statistic):               0.00
Time:                        20:08:44   Log-Likelihood:                -9929.5
No. Observations:                7000   AIC:                         1.987e+04
Df Residuals:                    6994   BIC:                         1.991e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.8825      0.033    118.335      0.0

# Bootstrapping Section

In [85]:
B = 10
coef_rows = []

for b in range(B):
    bootstrap = data.sample(10000, replace=True, random_state=33+b)
    model = smf.ols('y ~ x1 + x2 + x3 + x4 + x2_sq', data = bootstrap).fit()
    coef_rows.append(model.params.reindex(['Intercept','x1','x2','x3','x4','x2_sq']))

bootstrap_sample_coefs = pd.DataFrame(coef_rows)

bootstrap_summary = pd.DataFrame({'mean': bootstrap_sample_coefs.mean(), 'std': bootstrap_sample_coefs.std(ddof=1)})
print(bootstrap_summary)
print(bootstrap_sample_coefs)

               mean       std
Intercept  3.869277  0.023132
x1         0.814491  0.012286
x2        -0.013780  0.007550
x3         1.116420  0.045442
x4         1.620503  0.039919
x2_sq     -0.028671  0.006702
   Intercept        x1        x2        x3        x4     x2_sq
0   3.874732  0.807452 -0.017467  1.124180  1.605030 -0.019424
1   3.860256  0.817657 -0.006877  1.085329  1.691466 -0.025473
2   3.908564  0.822360 -0.018445  1.024926  1.662596 -0.034419
3   3.868613  0.821414 -0.021871  1.144689  1.595774 -0.026742
4   3.899209  0.814015  0.000433  1.106953  1.602773 -0.042509
5   3.871692  0.816652 -0.003668  1.158252  1.593825 -0.029991
6   3.823565  0.802218 -0.018343  1.161583  1.651153 -0.029746
7   3.864464  0.806602 -0.016706  1.079493  1.641703 -0.031182
8   3.862199  0.839980 -0.018905  1.171713  1.557499 -0.026771
9   3.859470  0.796559 -0.015951  1.107079  1.603207 -0.020459


# Conclusion

The coefficients when doing the validation is very similar to the coefficients when doing the bootstrapping. 